In [ ]:
import random
import os
import sys 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
from Bio import SeqIO
import gzip
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
random.seed(0)

In [ ]:
os.chdir("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137")

for dirname in ["table", "figures", "fasta"]:
    try:
        os.mkdir(dirname)
    except:
        None

In [ ]:
initial_sequence = ""
for _ in range(12):
    initial_sequence += random.choice(['A', 'T', 'G', 'C'])
for _ in range(9):
    for __ in range(3):
        initial_sequence += random.choice(['A', 'T', 'G', 'C'])
    for __ in range(15):
        initial_sequence += random.choice(['C', 'A'])
    for __ in range(7):
        initial_sequence += random.choice(['A', 'T', 'G', 'C'])
    for __ in range(2):
        initial_sequence += random.choice(['G']) # PAM: NGG
initial_sequence, len(initial_sequence) # This is not the sequence used in the study!


initial_sequence = "GGAGAAGTTGGCTGAACCACCAAAAACCAAGGACACCGGGTTACCCCCCACCAAAACCAGTGTCGGAAAACCAAACCCCCCAAATACTCCCGGCTTCAAACCACAAACCCCTTAGACGGGCGCCCCACAAAACCCCAAGACGGCCGGTTTAACCAACACCAACAACAACTAAGGAGTAACCAACACAACCCACGTCGAAGGGATCCAACCAACCACCACACCCACGGGGAAACAACCCACCCCCCCGGATTTCGG" # this seqeuence was the actually used sequence generated with an arbitrally picked random seed
initial_sequence, len(initial_sequence) # This is not the sequence used in the study!

In [ ]:
with open("fasta/initial.fasta", 'w') as handle:
    handle.write(">root\n")
    handle.write(initial_sequence + "\n")

In [ ]:
df = pd.read_table("node_count.txt", names = ('date', 'date_sec', 'running', 'running+waiting'))
df

In [ ]:
df['waiting'] = df['running+waiting'] - df['running']
df['run_time'] = df['date_sec'] - 1605251103
df

In [ ]:
fig = plt.figure(figsize=(3,2))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#sns.lineplot(data=df, x="run_time", y="running+waiting", label = 'running + waiting')
#sns.lineplot(data=df, x="run_time", y="running",label='running')
pal = sns.color_palette("Set1")

x=list(df['run_time'])
y=[list(df['running']), list(df['waiting'])]

plt.stackplot(x,y, labels=['Running','Waiting'], colors=pal, alpha=0.4 )
plt.legend(loc='upper right')

ax.set_xlabel("Time (t)")
ax.set_ylabel("# jobs")
ax.get_legend().remove()
plt.savefig("figures/time_node.pdf", bbox_inches='tight')
plt.close()

In [ ]:
df_25_result = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/results/result.FRACTAL_0.25.accuracy.csv", names = ['Nchunks','Ntips','NRFD'])
df_25_result['Coverage (%)'] = df_25_result['Ntips']/16671840*100
df_25_result['Accuracy (%)'] = (1-df_25_result['NRFD'])*100
df_25_result['Recovery ratio (%)'] = df_25_result['Coverage (%)']*df_25_result['Accuracy (%)']/100

df_025_result = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/results/result.FRACTAL_0.025.accuracy.csv", names = ['Nchunks','Ntips','NRFD'])
df_025_result['Coverage (%)'] = df_025_result['Ntips']/16671840*100
df_025_result['Accuracy (%)'] = (1-df_025_result['NRFD'])*100
df_025_result['Recovery ratio (%)'] = df_025_result['Coverage (%)']*df_025_result['Accuracy (%)']/100

In [ ]:
df_25_result.sort_values('Nchunks')

In [ ]:
min(df_025_result['Coverage (%)'])

In [ ]:
fig = plt.figure(figsize=(1.8,1.8))

ax1 = fig.add_axes([0.1,0.1,0.8,0.8])
ax2 = fig.add_axes([1.3,0.1,0.8,0.8])
for df, ax in [(df_25_result, ax1),(df_025_result, ax2)]:
    ax.scatter(x=df["Nchunks"], y=df["Accuracy (%)"], alpha = 0.5)
    ax.scatter(x=df["Nchunks"], y=df["Coverage (%)"], alpha = 0.5)
    ax.set_xlim(0,105)
    ax.set_ylim(75,101)
    ax.set_xlabel("# chunks")
    ax.set_ylabel("Accuracy or Coverage (%)")
plt.savefig("figures/fractal_fig1.pdf", bbox_inches='tight')
#plt.show()
plt.close()

fig = plt.figure(figsize=(1.8,1.5))

ax = fig.add_axes([0.1,0.1,0.8,0.8])

ax.scatter(x=df_25_result["Nchunks"],  y=df_25_result["Accuracy (%)"], alpha  = 1, s= 8, color = "#009051")
ax.scatter(x=df_025_result["Nchunks"], y=df_025_result["Accuracy (%)"], alpha = 1, s= 8, color = "#00F900")
#ax.scatter(x=df_25_result["Nchunks"],  y=df_25_result["Accuracy (%)"], alpha  = 1, s= 10, color = "#011993")
#ax.scatter(x=df_025_result["Nchunks"], y=df_025_result["Accuracy (%)"], alpha = 1, s= 10, color = "#FF9300")
#ax.scatter(x=df_25_result["Nchunks"],  y=df_25_result["Coverage (%)"], alpha  = 0.5, s= 10, edgecolors = "#009051", facecolors = 'none')
#ax.scatter(x=df_025_result["Nchunks"], y=df_025_result["Coverage (%)"], alpha = 0.5, s= 10, edgecolors = "#00FA92", facecolors = 'none')
ax.set_xlim(0,105)
ax.set_ylim(60,101)
ax.set_xlabel("# arrays")
ax.set_ylabel("Accuracy (%)")
plt.savefig("figures/NK_0137_fractal_fig2.pdf", bbox_inches='tight')
#plt.show()
plt.close()

In [ ]:
df_25_orthogonal = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/results/result.FRACTAL_0.25.orthogonal.csv", names = ['Nchunks','Ntips_ortho','NRFD_ortho'])
df_25_result = pd.merge(df_25_result, df_25_orthogonal)

df_025_orthogonal = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/results/result.FRACTAL_0.025.orthogonal.csv", names = ['Nchunks','Ntips_ortho','NRFD_ortho'])
df_025_result = pd.merge(df_025_result, df_025_orthogonal)
df_025_result

In [ ]:
df_25_result

In [ ]:
fig = plt.figure(figsize=(0.9,1.5))

ax = fig.add_axes([0.1,0.1,0.8,0.8])
ax.scatter(x=df_25_result["Nchunks"], y=(1-df_25_result["NRFD_ortho"])*100, alpha  = 1, s= 8, color = "#009051")
ax.scatter(x=df_025_result["Nchunks"], y=(1-df_025_result["NRFD_ortho"])*100, alpha = 1, s= 8, color = "#00F900")
#ax.scatter(x=df_25_result["Nchunks"], y=(1-df_25_result["NRFD_ortho"])*100, alpha  = 1, s= 10, color = "#011993")
#ax.scatter(x=df_025_result["Nchunks"], y=(1-df_025_result["NRFD_ortho"])*100, alpha = 1, s= 10, color = "#FF9300")
ax.set_xlim(0,52.5)
ax.set_xticks([0,25,50])
ax.set_ylim(60,101)
ax.set_xlabel("# arrays")
ax.set_ylabel("Agreement between\northogonal trees")
plt.savefig("figures/NK_0137_fractal_fig3.pdf", bbox_inches='tight')
#plt.show()
plt.close()

In [ ]:
fig = plt.figure(figsize=(1.5,1.5))

ax = fig.add_axes([0.1,0.1,0.8,0.8])
ax.scatter(x=(1-df_25_result["NRFD_ortho"])*100, y=df_25_result["Accuracy (%)"], alpha  = 1, s= 8, color = "#009051")
ax.scatter(x=(1-df_025_result["NRFD_ortho"])*100, y=df_025_result["Accuracy (%)"], alpha = 1, s= 8, color = "#00F900")
#ax.scatter(x=(1-df_25_result["NRFD_ortho"])*100, y=df_25_result["Accuracy (%)"], alpha  = 1, s= 10, color = "#011993")
#ax.scatter(x=(1-df_025_result["NRFD_ortho"])*100, y=df_025_result["Accuracy (%)"], alpha = 1, s= 10, color = "#FF9300")

df_concat_result = pd.concat([df_25_result, df_025_result])

lr = LinearRegression()
X = [[x] for x in (1-df_concat_result["NRFD_ortho"])*100]
Y = df_concat_result["Accuracy (%)"]
lr.fit(X, Y)  
coefficient = lr.coef_[0]
intercept = lr.intercept_

ax.plot([0,100], lr.predict([[0],[100]]), color = 'grey', alpha=0.5, linewidth = 1) 
print('coefficient = ', coefficient) 
print('intercept = ', intercept)
print('R^2 = ', r2_score(Y, lr.predict(X)))

ax.set_xlim(60,101)
ax.set_ylim(60,101)
#ax.set_xlim(0,0.3)
ax.set_xticks([60,80,100])
ax.set_xlabel('Agreement between\northogonal trees')
ax.set_ylabel("Accuracy (%)")
plt.savefig("figures/NK_0137_fractal_fig4.pdf", bbox_inches='tight')
#plt.show()
plt.close()

In [ ]:
# for fig 4f
df_a2g = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/Target-ACEmax_mu_values_A2G_edits.csv")
df_c2t = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/Target-ACEmax_mu_values_C2T_edits.csv")

In [ ]:
for header, df_edit, color in [('C2T', df_c2t, '#4575B3'), ('A2G', df_a2g, '#E4B846')]:
    site_relativefreq_list = [float(value) for value in df_edit.iloc[0,:][3:30]]
    #print(sum(site_relativefreq_list))
    fig = plt.figure(figsize=(2,2))
    ax = fig.add_axes([0.1,0.1,0.8,0.4])
    mu_list = np.array([float(value) for value in df_edit['mu value'][1:]])
    ax.hist(mu_list, bins = 10, range= (0,3), color = color, orientation='horizontal', density = True)
    ax.set_yticks([0, 1, 2])
    ax.set_ylim(0,3)
    ax.set_ylabel('Scale (x)')
    ax.set_xlabel('Frequency')
    plt.savefig("figures/"+header+"_NK_0137_mu_histogram.pdf", bbox_inches='tight')
    #plt.show()
    plt.close()

In [ ]:
df_orthogonal = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/orthogonal/result.csv", names=['scale','Nchunks','Nsample','Ntips','NRFD'])
df_orthogonal


In [ ]:
for scale in [0.25, 0.025]:
    for Nchunks in [15,18,20,30,40,50]:
        df_orthogonal_exp = df_orthogonal[(df_orthogonal['scale']==scale) & (df_orthogonal['Nchunks']==Nchunks)]
        df_orthogonal_exp

        fig = plt.figure(figsize=(2,2))
        ax = fig.add_axes([0.1,0.1,0.8,0.8])
        ax.scatter(df_orthogonal_exp['Ntips'],1-df_orthogonal_exp['NRFD'], color = 'black')
        ax.set_xscale('log')
        ax.set_ylim(0,1.05)
        ax.set_xlabel('#sequences')
        ax.set_ylabel('Agreement between orthogonal trees')
        ax.set_title(str(scale)+": "+str(Nchunks)+" chunks")
        plt.savefig("figures/fig_"+str(scale)+"_"+str(Nchunks)+"chunks.pdf", bbox_inches='tight')
        #plt.show()
        plt.close()

for scale in [0.25, 0.025]:
    for Nchunks in [15,18,20,30,40,50]:
        df_orthogonal_exp = df_orthogonal[(df_orthogonal['scale']==scale) & (df_orthogonal['Nchunks']==Nchunks)]
        df_orthogonal_exp

        fig = plt.figure(figsize=(2,2))
        ax = fig.add_axes([0.1,0.1,0.8,0.8])
        ax.scatter(df_orthogonal_exp['Ntips'],1-df_orthogonal_exp['NRFD'], color = 'black')
        ax.set_xscale('log')
        #ax.set_ylim(0,1.05)
        ax.set_xlabel('#sequences')
        ax.set_ylabel('1- NRFD')
        ax.set_title(str(scale)+": "+str(Nchunks)+" chunks")
        plt.savefig("figures/fig_"+str(scale)+"_"+str(Nchunks)+"chunks_zoom.pdf", bbox_inches='tight')
        #plt.show()
        plt.close()

In [ ]:
df_del_prob = pd.read_table("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/prob.del.0.17.txt", names=['deletion'])
df_ins_prob = pd.read_table("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/prob.ins.0.0075.txt", names=['deletion'])
df_del

In [ ]:
records = SeqIO.parse("example/initial.fasta",'fasta')
for record in records:
    initial_sequence = str(record.seq)
initial_sequence

In [ ]:
for fastafile in ["/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/chunk87_0.025.fa.gz", "/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/chunk34_0.25.fa.gz"]:
    #fastafile="/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/chunk34_0.25.fa.gz"
    count_list = np.zeros((255,5))
    char2idx = {'C':0,'T':1,'A':2,'G':3,'-':4}
    records= SeqIO.parse(gzip.open(fastafile,'rt'),'fasta')

    k=0
    for record in records:
        if(k%10000==0):print(k)
        for i in range(len(record.seq)):
            count_list[i][char2idx[str(record.seq)[i]]]+=1
        k+=1
    np.savetxt('/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/'+fastafile.split('/')[-1]+'.csv',count_list,delimiter=',')

In [ ]:
for csvfile in ["/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/chunk87_0.025.fa.gz.csv", "/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/chunk34_0.25.fa.gz.csv"]: 
    #csvfile="/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/chunk87_0.025.fa.gz.csv"
    df_pattern = pd.read_csv(csvfile, names=['C','T','A','G','-'])
    df_pattern['CtoT'] = [df_pattern['T'][i]/(df_pattern['C'][i] +df_pattern['T'][i]) if initial_sequence[i]=='C' else 0 for i in range(len(df_pattern))]
    df_pattern['AtoG'] = [df_pattern['G'][i]/(df_pattern['A'][i] +df_pattern['G'][i]) if initial_sequence[i]=='A' else 0 for i in range(len(df_pattern))]
    df_pattern['deletion'] = [df_pattern['-'][i]/16671840 for i in range(len(df_pattern))]
    
    df_pattern
    fig = plt.figure(figsize=(4.2,1.0))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    ax.bar(x = list(range(255)), height=df_pattern['CtoT'], color='#4575B3',width=1)
    ax.set_xlim(0,255)
    ax.set_ylim(0,1)
    ax.set_ylabel('Proportion')
    ax.set_xlabel('Position')
    ax.set_title(csvfile.split("/")[-1])
    plt.savefig("figures/NK_0137_CtoT_"+csvfile.split("/")[-1]+".pdf",bbox_inches='tight')

    fig = plt.figure(figsize=(4.2,1.0))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    ax.bar(x = list(range(255)), height=df_pattern['AtoG'], color='#E4B846',width=1)
    ax.set_xlim(0,255)
    ax.set_ylim(0,1)
    ax.set_ylabel('Proportion')
    ax.set_xlabel('Position')
    ax.set_title(csvfile.split("/")[-1])
    plt.savefig("figures/NK_0137_AtoG_"+csvfile.split("/")[-1]+".pdf",bbox_inches='tight')
    
    fig = plt.figure(figsize=(4.2,1.0))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    ax.bar(x = list(range(255)), height=df_pattern['deletion'], color="#FF0000",alpha=0.5,width=1)
    ax.set_xlim(0,255)
    ax.set_ylim(0,1)
    ax.set_ylabel('Proportion')
    ax.set_xlabel('Position')
    ax.set_title(csvfile.split("/")[-1])
    plt.savefig("figures/NK_0137_deletion_"+csvfile.split("/")[-1]+".pdf",bbox_inches='tight')

In [ ]:
count_list

In [ ]:
for indelfile in ["/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/chunk34_0.25.indel.gz", "/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/chunk87_0.025.indel.gz"]:
    inspos2count=np.zeros(255)
    with gzip.open(indelfile, 'rt') as handle:
        for line in handle:
            if (len(line.split())>1):
                name = line.split()[0]
                indels = line.split()[1].split(";")
                for indel in indels:
                    if(indel.split("_")[0]=='I'):
                        pos = int(float(indel.split("_")[1]))
                        inspos2count[pos] += 1
                        
    fig = plt.figure(figsize=(4.2,1.0))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    ax.bar(x = list(range(255)), height=inspos2count/16671840, color="#0000FF",alpha=0.5,width=1)
    ax.set_xlim(0,255)
    ax.set_ylim(0,0.018)
    ax.set_ylabel('Proportion')
    ax.set_xlabel('Position')
    ax.set_title(indelfile.split("/")[-1])
    plt.savefig("figures/NK_0137_insertion_"+indelfile.split("/")[-1]+".pdf",bbox_inches='tight')

In [ ]:
for indelfile in ["/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/chunk34_0.25.indel.gz", "/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/example/chunk87_0.025.indel.gz"]:
    delpos2count=np.zeros(255)
    with gzip.open(indelfile, 'rt') as handle:
        for line in handle:
            if (len(line.split())>1):
                name = line.split()[0]
                indels = line.split()[1].split(";")
                for indel in indels:
                    if(indel.split("_")[0]=='D'):
                        pos = int(indel.split("_")[1].split("mid")[1])
                        delpos2count[pos] += 1
                        
    fig = plt.figure(figsize=(4.2,1.0))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    ax.bar(x = list(range(255)), height=delpos2count/16671840, color='#FF0000',alpha=0.5,width=1)
    ax.set_xlim(0,255)
    ax.set_ylabel('Proportion')
    ax.set_xlabel('Position')
    ax.set_title(indelfile.split("/")[-1])
    plt.savefig("figures/NK_0137_deletion_"+indelfile.split("/")[-1]+".pdf",bbox_inches='tight')
    plt.close()

In [ ]:
# compare cassiopeia, fractal+cassiopeia, fractal+mp, and (fractal+nj)

def nearest(m, nlist):
    diff = abs(m-nlist[0])
    nearest = nlist[0]
    for n in nlist:
        if abs(m-n) < diff:
            nearest = n
            diff = abs(m-n)
    return nearest

df_cassiopeia = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/result_vs_cassiopeia/result.cassiopeia.csv", names = ["TASK_ID", "x", "Nchunks", "Nseq", "inputfile","newick", "method", "Memory", "Mem_unit", "Time", "Time_unit", "Ntips", "NRFD", "Triplet"])
df_cassiopeia["Coverage"] = 100

df_fractal_cassiopeia = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/result_vs_cassiopeia/result.fractal_cassiopeia.csv", names = ["TASK_ID", "x", "Nchunks", "Nseq", "inputfile", "newick", "method", "Memory", "Mem_unit", "Time", "Time_unit", "Ntips", "NRFD", "Triplet"])
df_fractal_cassiopeia["method"] = ["fractal_"+method for method in df_fractal_cassiopeia["method"]]

df_fractal_cassiopeia["Coverage"] = df_fractal_cassiopeia["Ntips"] / df_fractal_cassiopeia["Nseq"] * 100

df_fractal_raxmlMP = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0137/result_vs_cassiopeia/result.fractal_raxmlMP.csv", names = ["TASK_ID", "x", "Nchunks", "Nseq", "inputfile","newick", "method", "Memory", "Mem_unit", "Time", "Time_unit", "Ntips", "NRFD", "Triplet"])
df_fractal_raxmlMP["method"] = ["fractal_"+method for method in df_fractal_raxmlMP["method"]]
df_fractal_raxmlMP["Coverage"] = df_fractal_raxmlMP["Ntips"] / df_fractal_raxmlMP["Nseq"] * 100

df_vs_cassiopeia = pd.concat([df_cassiopeia, df_fractal_cassiopeia, df_fractal_raxmlMP]).reindex(columns= ["TASK_ID", "x", "Nchunks", "Nseq", "inputfile", "newick", "method", "Memory", "Mem_unit", "Time", "Time_unit", "Ntips", "NRFD", "Triplet", "Coverage"])

df_vs_cassiopeia["Accuracy"] = (1-df_vs_cassiopeia["NRFD"]) * 100
df_vs_cassiopeia["Triplet_percent"] = df_vs_cassiopeia["Triplet"] * 100
#df_vs_cassiopeia["Coverage"] = df_vs_cassiopeia["Ntips"] / df_vs_cassiopeia["Nseq"] * 100

df_vs_cassiopeia["Recovery_NRFD"] = (1-df_vs_cassiopeia["NRFD"]) * df_vs_cassiopeia["Coverage"]
df_vs_cassiopeia["Recovery_Triplet"] = (df_vs_cassiopeia["Triplet"]) * df_vs_cassiopeia["Coverage"]
df_vs_cassiopeia["nearestNseq"]   = [nearest(Nseq, [100, 1000, 10000]) for Nseq in df_vs_cassiopeia["Nseq"]]

for column in ["Memory", "Coverage", "Time"]:
    df_vs_cassiopeia[column] = [list(df_vs_cassiopeia[column])[i] if not np.isnan(list(df_vs_cassiopeia["NRFD"])[i]) else np.nan for i in range(len(df_vs_cassiopeia[column]))]

df_vs_cassiopeia.to_csv("table/result.csv")
df_vs_cassiopeia

In [ ]:
method_list = ["greedy", "hybrid", "ilp", "fractal_greedy", "fractal_hybrid", "fractal_ilp", "fractal_raxmlMP", "fractal_rapidnjNJ"]


Colors = [ "#7BACD2","#E5924E",  "#87C37E", \
           "#4C81B9", "#D55F2B", "#56A15D", 
           "#3D33FF", "#4996FF"
          ]

for Nseq in [100, 1000, 10000]:
    for x in [0.25, 0.025, 0]:
        for target, ylabel, ymax in [("Accuracy", "Accuracy (%)", 100), ("Triplet_percent", "Triplet correct (%)", 100), ("Recovery_NRFD", "Recovery ratio (%)", 100), ("Recovery_Triplet", "Recovery ratio (%)", 100), ("Time", "log$_{10}$ Run Time (sec)", 86400), ("Memory", "log$_{10}$ Memory (KB)", 16000000), ("Coverage", "Coverage (%)", 100)]:

            #Nseq = 100
            #x    = 0.25
            #target = "Recovery"
            #ylabel = "Recovery ratio (%)"

            barWidth= 0.1

            #for name, figsize in [("short", (2.3,1.5)), ("long", (4, 1.5))]: 
            for name, figsize in [("short", (2.3,1.5))]: 

                fig = plt.figure(figsize=figsize)
                ax  = fig.add_axes([0.1,0.1,0.8,0.8])

                df_vs_cassiopeia_ext = df_vs_cassiopeia[(df_vs_cassiopeia["nearestNseq"]==Nseq) & (df_vs_cassiopeia["x"]==x)]

                for i, method in enumerate(method_list):

                    if (target == "Memory" or target == "Time"):

                        bars   = list(np.log10(df_vs_cassiopeia_ext[df_vs_cassiopeia_ext["method"]==method][target]))
                    
                    else:

                        bars   = list(df_vs_cassiopeia_ext[df_vs_cassiopeia_ext["method"]==method][target])
                                
                    r   = np.arange(len(bars))
                    r_i = [x + barWidth*i for x in r]

                    ax.bar(r_i, bars, width = barWidth, color = Colors[i], label=method)

                #if (target == "Memory" or target == "Time"):
                #    ax.set_yscale("log")

                if (target == "Memory"):
                    ax.set_ylim(np.log10(10000),np.log10(16000000))
                elif (target == "Time"):
                    #ax.set_ylim(np.log10(1),np.log10(86400))
                    ax.set_ylim(np.log10(1),5)
                else:
                    ax.set_ylim(0,ymax)
                ax.set_xlabel("Number of arrays")
                ax.set_ylabel(ylabel)
                ax.set_title(target+" "+str(x)+"x "+str(Nseq)+" seq", fontsize=5)

                plt.xticks([r + barWidth*3.5 for r in range(len(bars))], [5, 10, 20, 40, 100])

                plt.savefig("figures/NK_0137_vs_cassiopeia_"+target+"_"+str(x)+"_"+str(Nseq)+"_"+name+".pdf", bbox_inches='tight')
                plt.close()

In [ ]:
method_list = ["greedy", "hybrid", "ilp", "fractal_greedy", "fractal_hybrid", "fractal_ilp", "fractal_raxmlMP", "fractal_rapidnjNJ"]


Colors = [ "#7BACD2","#E5924E",  "#87C37E", \
           "#4C81B9", "#D55F2B", "#56A15D", 
           "#3D33FF", "#4996FF"
          ]

for Nseq in [100]:
    for x in [0.25]:
        for target, ylabel, ymax in [("Accuracy", "Accuracy (%)", 100), ("Triplet_percent", "Triplet correct (%)", 100), ("Recovery_NRFD", "Recovery ratio (%)", 100), ("Recovery_Triplet", "Recovery ratio (%)", 100), ("Time", "log$_{10}$ Run Time (sec)", 86400), ("Memory", "log$_{10}$ Memory (KB)", 16000000), ("Coverage", "Coverage (%)", 100)]:

            #Nseq = 100
            #x    = 0.25
            #target = "Recovery"
            #ylabel = "Recovery ratio (%)"

            barWidth= 0.1

            #for name, figsize in [("short", (2.3,1.5)), ("long", (4, 1.5))]: 
            for name, figsize in [("short", (2.3,1.5))]: 

                fig = plt.figure(figsize=figsize)
                ax  = fig.add_axes([0.1,0.1,0.8,0.8])

                df_vs_cassiopeia_ext = df_vs_cassiopeia[(df_vs_cassiopeia["nearestNseq"]==Nseq) & (df_vs_cassiopeia["x"]==x)]

                for i, method in enumerate(method_list):

                    #if (target == "Memory" or target == "Time"):

                    #    bars   = list(np.log10(df_vs_cassiopeia_ext[df_vs_cassiopeia_ext["method"]==method][target]))
                    
                    #else:

                    bars   = list(df_vs_cassiopeia_ext[df_vs_cassiopeia_ext["method"]==method][target])
                                
                    r   = np.arange(len(bars))
                    r_i = [x + barWidth*i for x in r]

                    ax.bar(r_i, bars, width = barWidth, color = Colors[i], label=method)

                if (target == "Memory" or target == "Time"):
                    ax.set_yscale("log")

                if (target == "Memory"):
                    ax.set_ylim(10000,16000000)
                elif (target == "Time"):
                    #ax.set_ylim(np.log10(1),np.log10(86400))
                    ax.set_ylim(1,100000)
                else:
                    ax.set_ylim(0,ymax)
                ax.set_xlabel("Number of arrays")
                ax.set_ylabel(ylabel)
                ax.set_title(target+" "+str(x)+"x "+str(Nseq)+" seq", fontsize=5)

                plt.xticks([r + barWidth*3.5 for r in range(len(bars))], [5, 10, 20, 40, 100])

                plt.savefig("figures/NK_0137_vs_cassiopeia_"+target+"_"+str(x)+"_"+str(Nseq)+"_"+name+"_old"+".pdf", bbox_inches='tight')
                plt.close()